In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# Function to parse each line of the file
def parse_line(line):
    # Split the line by commas for the titles or by spaces for the data
    parts = line.strip().split(',') if ',' in line else line.strip().split()
    return parts

# Specify the names of the files to search for
target_files = ['flow_cut_to_failure.vtk', 'transport_cut_to_failure.vtk']
n = 90

# Arrays to store the read data
h2aq = np.zeros([n+1, 1])
h2saq = np.zeros([n+1, 1])
h2g = np.zeros([n+1, 1])

In [9]:
for i in range(1,n+1):
    # Get the current directory
    current_dir = './../Hem_Base_2/run_' + str(i) + '/'

    # Search for the target files in the current directory
    for file in os.listdir(current_dir):
        if file in target_files:
            print("Failure found in folder:", current_dir)
            break  # Exit the loop if either file is found
        else:
            read_folder = True
            
    if not os.path.exists('./../Hem_Base_2/run_' + str(i) + '/pflotran_h2-012.vtk'):
            print(f"File does not exist: {'./../Hem_Base_2/run_' + str(i) + '/pflotran_h2-012.vtk'}")
            continue
    else:
        file_path_h2 = './../Hem_Base_2/run_' + str(i) + '/pflotran_h2-012.vtk'
        #============
        # Read H2(aq)
        # ============
        # Initialize an empty list to store the numbers
        numbers_list_h2 = []
        numbers_list_sat = []

        # Define a flag to identify when to start reading numbers
        start_reading_h2 = False
        start_reading_sat = False

        # Open the file and read line by line
        with open(file_path_h2, 'r') as file:
            for line in file:
                # Check if the line contains the title indicating the start of numbers
                if 'SCALARS Total_H2(aq)' in line:
                    start_reading_h2 = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_h2:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_h2 = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_h2.extend([float(num) for num in line.split()])

                if 'SCALARS Liquid_Saturation' in line:
                    start_reading_sat = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_sat:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_sat = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_sat.extend([float(num) for num in line.split()])

        # Convert the list to a numpy array
        numbers_array_h2_all = np.array(numbers_list_h2)
        numbers_array_h2 = numbers_array_h2_all[numbers_array_h2_all <= 1.0]
        numbers_array_sat_all = np.array(numbers_list_sat)
        numbers_array_sat = numbers_array_sat_all[numbers_array_h2_all <= 1.0]

        cell_vol_gr = 16*16*20*0.2*1000

        # Display the first few numbers to verify
        cell_h2 = cell_vol_gr*numbers_array_sat*numbers_array_h2
        total_h2 = np.sum(cell_h2)

        # print("Total H2(aq) is " + str(total_h2) + " moles")
        h2aq[i, 0] = total_h2

        # ============
        # Read H2S(aq)
         # ============
        file_path_h2s = './../Hem_Base_2/run_' + str(i) + '/pflotran_h2-012.vtk'
        # Initialize an empty list to store the numbers
        numbers_list_h2s = []
        numbers_list_sat = []

        # Define a flag to identify when to start reading numbers
        start_reading_h2s = False
        start_reading_sat = False

        # Open the file and read line by line
        with open(file_path_h2s, 'r') as file:
            for line in file:
                # Check if the line contains the title indicating the start of numbers
                if 'SCALARS H2S(aq)' in line:
                    start_reading_h2s = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_h2s:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_h2s = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_h2s.extend([float(num) for num in line.split()])

                if 'SCALARS Liquid_Saturation' in line:
                    start_reading_sat = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_sat:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_sat = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_sat.extend([float(num) for num in line.split()])

        # Convert the list to a numpy array
        numbers_array_h2s_all = np.array(numbers_list_h2s)
        numbers_array_h2s = numbers_array_h2s_all[(numbers_array_h2s_all <= 1.0) & (numbers_array_h2s_all >= 2e-8)]
        # numbers_array_sat_all = np.array(numbers_list_sat)
        # numbers_array_sat = numbers_array_sat_all[numbers_array_h2s_all <= 1.0 and numbers_array_h2s_all >= 2e-8]

        # cell_vol_gr = 16*16*20*0.2*1000

        # # Display the first few numbers to verify
        # cell_h2s = cell_vol_gr*numbers_array_sat*numbers_array_h2s
        # total_h2s = np.sum(cell_h2s)

        # print("Total H2S(aq) is: " + str(total_h2s) + " moles")
        h2saq[i, 0] = np.average(numbers_array_h2s) # total_h2s

        # ==========
        # Read H2(g)
        # ==========
        # Define the path to your .dat file
        file_path_h2g = './../Hem_Base_2/run_' + str(i) + '/pflotran_h2-mas.dat'

        # Read the file and parse each line
        with open(file_path_h2g, 'r') as file:
            lines = file.readlines()

        # Parse the column titles
        column_titles = parse_line(lines[0])

        # Initialize a list to hold the parsed data
        data = []

        # Parse each line of data
        for line in lines[1:]:  # Skip the first line with column titles
            parsed_line = parse_line(line)
            if parsed_line:  # Ensure the line is not empty
                data.append(parsed_line)

        # Create a DataFrame from the parsed data
        # The length of the longest row is used to ensure all rows have the same length
        max_length = max(len(row) for row in data)
        df = pd.DataFrame(data, columns=column_titles[:max_length])

        df.columns = df.columns.str.strip('"')
        last_number_global_h2 = df["Global H2(g) [mol]"].iloc[11]
        last_number_global_h2 = pd.to_numeric(last_number_global_h2, errors='coerce')
        h2g[i, 0] = last_number_global_h2
        print("The H2(g) [mol] after 1Y is: ", last_number_global_h2)


The H2(g) [mol] after 1Y is:  90566662600.0
The H2(g) [mol] after 1Y is:  90390812800.0
The H2(g) [mol] after 1Y is:  90093549500.0
The H2(g) [mol] after 1Y is:  89850654500.0
The H2(g) [mol] after 1Y is:  89660496700.0
The H2(g) [mol] after 1Y is:  89465093200.0
The H2(g) [mol] after 1Y is:  89177896300.0
The H2(g) [mol] after 1Y is:  89062039100.0
The H2(g) [mol] after 1Y is:  89000145100.0
The H2(g) [mol] after 1Y is:  88958371500.0
The H2(g) [mol] after 1Y is:  88929006400.0
The H2(g) [mol] after 1Y is:  88907160100.0
The H2(g) [mol] after 1Y is:  88890249500.0
The H2(g) [mol] after 1Y is:  88876758900.0
The H2(g) [mol] after 1Y is:  88865738200.0
The H2(g) [mol] after 1Y is:  90566511800.0
The H2(g) [mol] after 1Y is:  90229516700.0
The H2(g) [mol] after 1Y is:  89731608000.0
The H2(g) [mol] after 1Y is:  89442838100.0
The H2(g) [mol] after 1Y is:  89285820100.0
The H2(g) [mol] after 1Y is:  89152429100.0
The H2(g) [mol] after 1Y is:  88967533000.0
The H2(g) [mol] after 1Y is:  88

In [10]:
print(h2aq)
print(h2saq)
print(h2g)

np.savetxt('./H2AQ_hem_Base_2_1Y.txt', h2aq, fmt='%d', delimiter=', ')
np.savetxt('./H2SAQ_hem_Base_2_1Y.txt', h2saq, fmt='%.10e', delimiter=', ')
np.savetxt('./H2G_hem_Base_2_1Y.txt', h2g, fmt='%d', delimiter=', ')

[[0.00000000e+00]
 [7.57625260e+08]
 [7.56160523e+08]
 [7.53663039e+08]
 [7.51621193e+08]
 [7.50022710e+08]
 [7.48380238e+08]
 [7.45965954e+08]
 [7.44991547e+08]
 [7.44464591e+08]
 [7.44113218e+08]
 [7.43866180e+08]
 [7.43682378e+08]
 [7.43540084e+08]
 [7.43426560e+08]
 [7.43333813e+08]
 [7.57637598e+08]
 [7.54813619e+08]
 [7.50628124e+08]
 [7.48200781e+08]
 [7.46880883e+08]
 [7.45759345e+08]
 [7.44203736e+08]
 [7.43581426e+08]
 [7.43239376e+08]
 [7.43021680e+08]
 [7.42870538e+08]
 [7.42759262e+08]
 [7.42673839e+08]
 [7.42606146e+08]
 [7.42551151e+08]
 [7.46067959e+08]
 [7.45311803e+08]
 [7.43979220e+08]
 [7.42865033e+08]
 [7.41953310e+08]
 [7.40959941e+08]
 [7.39385854e+08]
 [7.38710204e+08]
 [7.38316386e+08]
 [7.38053438e+08]
 [7.37863814e+08]
 [7.37719883e+08]
 [7.37606636e+08]
 [7.37515051e+08]
 [7.37439388e+08]
 [7.84469467e+08]
 [7.78926594e+08]
 [7.71324631e+08]
 [7.67636528e+08]
 [7.65790966e+08]
 [7.64281499e+08]
 [7.62301587e+08]
 [7.61560761e+08]
 [7.61168760e+08]
 [7.609252